In [2]:
import pandas as pd

df = pd.read_csv("deepseek.csv")
df.head()

,Punctuation Type,Sentence (Wrote),Sentence (Meant),DeepSeek
0,Comma,Chanting the choir raised the volume as the ce...,"Chanting, the choir raised the volume as the c...",गायन करणारा गायनसमूह जसजशा उत्सवाध्यक्षाने प्र...
1,Comma,A six-month-old calf was submitted for examina...,A six-month-old calf was submitted for examina...,एका सहा-महिन्यांच्या वासरूची तपासणीसाठी सादर क...
2,Comma,Planning authorities should provide alternativ...,Planning authorities should provide alternativ...,नियोजन प्राधिकरणांनी लघुव्यवसायांसाठी पर्यायी ...
3,Comma,As the machine develops the forms we use to re...,"As the machine develops, the forms we use to r...","जसजशी मशीन विकसित होते, तसतसे मागील प्रकल्पांम..."
4,Comma,"As mentioned, first impressions can be mislead...","As mentioned first, impressions can be mislead...","जसे उल्लेख केला आहे, पहिल्या छापी गैरसमज निर्म..."


In [3]:
translations = [sent.strip() for sent in df['DeepSeek']]
translations

['गायन करणारा गायनसमूह जसजशा उत्सवाध्यक्षाने प्रार्थना म्हटली तसतसे आवाज वाढवत होता.',
 'एका सहा-महिन्यांच्या वासरूची तपासणीसाठी सादर करण्यात आली, ज्याला जन्मानंतर लवकरच सुरू झालेल्या चारही पायांमध्ये लंगडेपणाची लक्षणे दिसून आली.',
 'नियोजन प्राधिकरणांनी लघुव्यवसायांसाठी पर्यायी स्थाने उपलब्ध करून द्यावीत, जे राहाण्याच्या भागात आक्षेपार्ह आहेत किंवा असू शकतात.',
 'जसजशी मशीन विकसित होते, तसतसे मागील प्रकल्पांमधून डेटा नोंदण्यासाठी आपण वापरत असलेली फॉर्मे सुधारली जातील.',
 'जसे उल्लेख केला आहे, पहिल्या छापी गैरसमज निर्माण करू शकतात.',
 'स्वच्छ असेंब्ली मिळवण्यासाठी, असेंब्ली चालू होण्यापूर्वी असेंब्ल्ड इक्वल्स टेबल लोड करा.',
 'कार्यकारी लोक मान्यताप्राप्त तारखांमधून मोठ्या विचलनाबद्दल माहिती देण्यास विलंब करतात; यामुळे वेळेत कारवाई करता येत नाही.',
 'हे ग्लायकन्स प्रोटीन्सवर खूपच कमी प्रमाणात स्थानांतरित केले जातात, ज्यामुळे ग्लायकोसिलेशन सीक्वॉन रिकामे राहतात.',
 'X हे मायग्रेनसाठी एक प्रभावी तीव्र, तोंडद्वारे घेण्याचे उपचार आहे ज्याची क्रिया झपाट्याने सुरू होते.',
 'माझ्या तज्ञ मतानु

In [6]:
from datasets import load_dataset
raw_datasets = load_dataset("thenlpresearcher/test_data_marathi")

In [7]:
src_sentences = raw_datasets['test']["sent_written"]
ref_indic     = raw_datasets['test']["indictrans2_en_indic_pe"]
ref_gt    = raw_datasets['test']["gt"]
ref_cfilt  = raw_datasets['test']["cfilt_model"]

In [8]:
mode = "deepseek"

In [12]:
import pandas as pd
from evaluate import load

# -------------------- SAVE OUTPUTS --------------------
results_df = pd.DataFrame({
    "src": src_sentences,
    "prediction": translations,
    "indic": ref_indic,
    "gt": ref_gt,
    "cfilt": ref_cfilt
})

results_df.to_csv(f"shalaka_{mode}_outputs.csv", index=False)
print(f"✔ Saved predictions to shalaka_{mode}_outputs.csv")

# -------------------- METRICS --------------------
bleu = load("sacrebleu")
chrf = load("chrf")

def compute_scores(preds, ref1, ref2, ref3):
    """
    Compute BLEU and chrF++ scores using all three references for each sentence.
    """
    references = [[r1, r2, r3] for r1, r2, r3 in zip(ref1, ref2, ref3)]  # sacrebleu format
    bleu_score = bleu.compute(predictions=preds, references=references)["score"]
    chrf_score = chrf.compute(predictions=preds, references=references)["score"]
    chrf2_score = chrf.compute(predictions=preds, references=references, char_order=6, word_order=2,  beta=2)["score"]
    return bleu_score, chrf_score, chrf2_score

bleu_score, chrf_score, chrf2_score = compute_scores(translations, ref_indic, ref_gt, ref_cfilt)

# Determine best reference per metric (based on BLEU)
all_scores = {
    "GT":    bleu.compute(predictions=translations, references=[[r] for r in ref_indic])["score"],
    "Gemini": bleu.compute(predictions=translations, references=[[r] for r in ref_gt])["score"],
    "CFILT":  bleu.compute(predictions=translations, references=[[r] for r in ref_cfilt])["score"]
}

best_ref = max(all_scores, key=all_scores.get)

print("\n===== FINAL METRICS =====")
print(f"All references combined → BLEU: {bleu_score:.2f}, chrF++: {chrf_score:.2f}, chrF2++: {chrf2_score:.2f}")
print(f"GT Marathi → BLEU: {all_scores['GT']:.2f}")
print(f"Gemini    → BLEU: {all_scores['Gemini']:.2f}")
print(f"CFILT     → BLEU: {all_scores['CFILT']:.2f}")
print(f"\n🎯 BEST REFERENCE = {best_ref} (by highest BLEU)")

# -------------------- SAVE METRICS --------------------
with open(f"shalaka_punct_{mode}_indictrans2_eval_metrics.txt", "w") as f:
    f.write(f"All references combined → BLEU {bleu_score:.2f}, chrF++ {chrf_score:.2f}, chrF2++: {chrf2_score:.2f}\n")
    f.write(f"GT    BLEU {all_scores['GT']:.2f}\n")
    f.write(f"Gem   BLEU {all_scores['Gemini']:.2f}\n")
    f.write(f"CFILT BLEU {all_scores['CFILT']:.2f}\n")
    f.write(f"\nBEST REFERENCE = {best_ref}\n")

print(f"Metrics written to shalaka_punct_{mode}_indictrans2_eval_metrics.txt")

✔ Saved predictions to shalaka_deepseek_outputs.csv

===== FINAL METRICS =====
All references combined → BLEU: 39.76, chrF++: 66.00, chrF2++: 62.85
GT Marathi → BLEU: 24.41
Gemini    → BLEU: 23.41
CFILT     → BLEU: 25.38

🎯 BEST REFERENCE = CFILT (by highest BLEU)
Metrics written to shalaka_punct_deepseek_indictrans2_eval_metrics.txt
